In [1]:
import brian2
from brian2tools import *
from brian2 import *
from struct import unpack
from PIL import Image
import numpy as np
import matplotlib.cm as cmap
import matplotlib.pyplot as plt
import os.path
import pickle as pickle
import scipy
import scipy.signal as signal
import time
import warnings
warnings.simplefilter('ignore')
import gc

In [2]:
def get_matrix_from_file(fileName, shape):
    readout = np.load(fileName)
#     print(readout.shape, fileName)
    value_arr = np.zeros(shape)
    if not readout.shape == (0,):
        value_arr[np.int32(readout[:,0]), np.int32(readout[:,1])] = readout[:,2]
    return value_arr

In [3]:
def gaussian2D(x, y, sigma):
    return (1.0/(1*math.pi*(sigma**2)))*math.exp(-(1.0/(sigma**2))*(x**2 + y**2))

def mexicanHat(x,y,sigma1,sigma2): 
    return gaussian2D(x,y,sigma1) - gaussian2D(x,y,sigma2)

def receptiveFieldMatrix(func):
    h = 5
    g = np.zeros((h,h))
    for xi in range(0,h):
        for yi in range(0,h):
            x = xi-int(h/2)
            y = yi-int(h/2)
            g[xi, yi] = func(x,y);
    return g

In [4]:
def tune_stimuli(data, r, x):
    data = (data - data.min()) / (data.max() - data.min())
    q95 = np.percentile(data, 95)
    data = np.where((data > q95), q95, data)
    q50 = np.percentile(data, 50)
    data = np.where((data < q50), q50, data)
    if (r!=0):
        data = np.exp(data*r)
    data = (data - data.min()) / (data.max() - data.min())
    data *= x
    return data


def get_stimuli(file_name, r1, r2, x):
    img = Image.open(file_name)
    img.load()
    img_arr = np.asarray(img, dtype="int32")
    stimuli = np.absolute(img_arr)/255
    img.close()
    print(stimuli.shape)
    
    stimuli_on = signal.convolve(stimuli, receptiveFieldMatrix(lambda x,y:mexicanHat(x,y,1,1.1)), mode='same')
    stimuli_on = tune_stimuli(stimuli_on, r1, x)
    stimuli_off = signal.convolve(stimuli, receptiveFieldMatrix(lambda x,y:mexicanHat(x,y,1.1,1)), mode='same')
    stimuli_off = tune_stimuli(stimuli_off, r2, x)

    c, axarr = subplots(1, 3, figsize = (15, 6))
    axarr[0].imshow(stimuli, cmap = cmap.get_cmap('binary'))
    axarr[0].title.set_text('stimuli')
    axarr[1].imshow(stimuli_on, cmap = cmap.get_cmap('binary'))
    axarr[1].title.set_text('stimuli_on')
    axarr[2].imshow(stimuli_off, cmap = cmap.get_cmap('binary'))
    axarr[2].title.set_text('stimuli_off')
    
    return stimuli_on, stimuli_off

In [ ]:
def save_connections():
    for connName in connections:
        conn = connections[connName]
        weights = np.column_stack((conn.i, conn.j, conn.w))
        sparseWeights = weights[~(weights.transpose()[2] == 0)]
        np.save(saved_path + connName, sparseWeights)
        print(connName, end=' ')
    print('connections saved')

In [ ]:
def get_surrounding(coordinate, r, inner, length):
    y = coordinate[0]
    x = coordinate[1]
    
    x_min = 0 if (x-r < 0) else (x-r)
    x_max = (length-1) if (x+r > length-1) else (x+r)
    y_min = 0 if (y-r < 0) else (y-r)
    y_max = (length-1) if (y+r > length-1) else (y+r)
    
    coordinates = []
    for i in range(y_min, y_max+1):
        for j in range(x_min, x_max+1):
            if (i==y and j==x and not inner):
                continue
            coordinates.append((i,j))
            
    return coordinates

def get_prefered_surrounding(coordinate, o_index, c_length):
    x = coordinate[0]
    y = coordinate[1]
    
    arr = []
    if (o_index == h_index):
        arr = [(x,y-2), (x,y-1), (x,y+1), (x,y+2)]
    elif (o_index == v_index):
        arr = [(x-2,y), (x-1,y), (x+1,y), (x+2,y)]
    elif (o_index == d1_index):
        arr = [(x+2,y-2), (x+1,y-1), (x-1,y+1), (x-2,y+2)]
    elif (o_index == d2_index):
        arr = [(x+2,y+2), (x+1,y+1), (x-1,y-1), (x-2,y-2)]
    
    coordinates = []
    for (p,q) in arr:
        if ((p>-1 and p<c_length) and (q>-1 and q<c_length)):
            coordinates.append((p,q))
    return coordinates

In [ ]:
def plot_lateral_distribution(name):
    weights = np.array(connections[name].w)
    weights = weights.reshape(shape)
    
    # init dictionary
    list_dic = {}; sum_dic = {}; total_sum = 0
    
    c = {0:'h', 1:'v', 2:'d1', 3:'d2'}
    for s_index in range(orientations):
        for t_index in range(orientations):
            list_dic[c[s_index]+c[t_index]] = []
            sum_dic[c[s_index]+c[t_index]] = 0

    for i in range(0, c_length):                   # y axis
        for j in range(0, c_length):               # x axis
            for p,q in get_surrounding((i,j), lateral_range, False, c_length):
                
                for s_index in range(orientations):
                    for t_index in range(orientations):
                        target = ( p * c_length + q ) * orientations + t_index
                        source = ( i * c_length + j ) * orientations + s_index
                        list_dic[c[s_index]+c[t_index]].append(weights[source][target])
                        sum_dic[c[s_index]+c[t_index]] += abs(weights[source][target])
                        total_sum += abs(weights[source][target])
                        
    category_names = ['H', 'V', 'Diagonal (45)', 'Diagonal (135)']
    results = { 'h': [], 'v': [], 'd1': [], 'd2': [] }
    %run horizontal_barchart_distribution.py
                        
    plt.figure(figsize=(17,4))
    for s_index in range(orientations):
        for t_index in range(orientations):
            plt.plot(list_dic[c[s_index]+c[t_index]], label=c[s_index]+c[t_index])
            
            print(c[s_index]+c[t_index], sum_dic[c[s_index]+c[t_index]], 
                  'of', total_sum, ':', sum_dic[c[s_index]+c[t_index]]*100/total_sum)
            
            results[c[s_index]].append(sum_dic[c[s_index]+c[t_index]])
    
    plt.legend(); plt.show()
    survey(results, category_names)

In [ ]:
def plot_preferred_distribution():
    c = {0:'horizontal', 1:'vertical', 2:'diagonal 45', 3:'diagonal 135'}
    shape = (n_L3, n_L3)
    weights = np.array(connections['L3_L3'].w)
    weights = weights.reshape(shape)
    
    # init dictionary
    w_arr = [[], [], [], []]
    
    for i in range(0, c_length):                   # y axis
        for j in range(0, c_length):               # x axis
            for o_index in range(orientations):
                for p,q in get_prefered_surrounding((i,j), o_index, c_length):
                    target = ( p * c_length + q ) * orientations + o_index
                    source = ( i * c_length + j ) * orientations + o_index
                    w_arr[o_index].append(weights[source][target])
                        
    plt.figure(figsize=(17,12))
    for o_index in range(orientations):
        plt.subplot(4, 1, o_index+1)
        plt.plot(w_arr[o_index], label=c[o_index])
        plt.legend()
        print(c[o_index], ' - ', sum(w_arr[o_index]))
    
    plt.show()
    gc.collect()

In [ ]:
def plot_integration(coordinates):
    c = ['horizontal', 'vertical', 'diagonal 45', 'diagonal 135']
    shape = (c_length, c_length, orientations)
    
    grid = np.array(spike_monitors['L3'].count).reshape(shape).transpose()
    plt.figure(figsize=(15,2))
    for o_index in range(orientations):
        arr = []
        for (i,j) in coordinates:
            arr.append(grid[o_index].transpose()[i,j])
        plt.plot(arr, label=c[o_index])
    plt.legend(); plt.show()

    grid = np.array(spike_monitors['L4'].count).reshape(shape).transpose()
    plt.figure(figsize=(15,2))
    for o_index in range(orientations):
        arr = []
        for (i,j) in coordinates:
            arr.append(grid[o_index].transpose()[i,j])
        plt.plot(arr, label=c[o_index])
    plt.legend(); plt.show()

# INITIALIZE

In [ ]:
stimuli_file_path = './pic_44_2.tif'

initial_path = './initial_weights/'
saved_path = './saved_weights/'

fig_num = 1

field_size = 5                     # 2/3/4/5/6
margin = (field_size//2)

c_length = 20
r_length = c_length*2 + margin*2
orientations = 4

h_index = 0
v_index = 1
d1_index = 2
d2_index = 3

n_LGN = r_length*r_length
n_L4  = c_length*c_length*orientations
n_L3  = n_L4
n_L2  = n_L3

lateral_range = 2

num_epochs = 12

single_example_time = 0.45 * second
resting_time = 0.25 * second

# delay = {}
# delay = (0*ms, 5*ms)              # min and max delay
# minDelay = delay[0]
# maxDelay = delay[1]
# deltaDelay = maxDelay - minDelay

# neural model parameters
v_rest_e = -65. * mV
v_rest_i = -60. * mV
v_reset_e = -65. * mV
v_reset_i = -45. * mV
v_thresh_e = -52 * mV
v_thresh_i = -40. * mV
refrac_e = 20. * ms
refrac_i = 15. * ms
tc_theta = 1e7 * ms
theta_plus_e = 0.2 * mV
offset = 20.0 * mV

# STDP parameters
tc_pre = 50*ms
tc_post = 40*ms
nu_pre =  0.05# 0.0001
nu_post = 0.1 # 0.01
wmax = 100.0
Apre = 0.4
Apost = Apre*1.05

input_intensity = 1.
start_input_intensity = input_intensity

#membrane dynamics
scr_e = 'v = v_reset_e; theta += theta_plus_e; timer = 0*ms'
v_reset_i_str = 'v = v_reset_i'

v_thresh_e_str = '(v > (theta - offset + v_thresh_e)) and (timer > refrac_e)'
v_thresh_i_str = 'v > v_thresh_i'

neuron_eqs_e = '''
        dv/dt = ((v_rest_e - v) + g_e*(-v) + g_i*(-100.*mV - v) ) / (100*ms)  : volt (unless refractory)
        dg_e/dt = -g_e/(1.0*ms)                                    : 1
        dg_i/dt = -g_i/(2.0*ms)                                    : 1
        dtheta/dt = -theta/(1e7*ms)                                : volt
        dtimer/dt = 0.1                                            : second
'''

neuron_eqs_i = '''
        dv/dt = ((v_rest_i - v) +  g_e*(-v) + g_i*(-85.*mV - v)) / (10*ms)  : volt (unless refractory)
        dg_e/dt = -g_e/(1.0*ms)                                    : 1
        dg_i/dt = -g_i/(2.0*ms)                                    : 1
'''

# learning rules
# without STDP
model = 'w : 1'
pre_e = 'g_e_post += w'
pre_i = 'g_i_post += w'
post = ''

stdp_model = '''
    post2_temp                          : 1
    w                                   : 1
    dpre/dt   =   -pre/(tc_pre)         : 1 (event-driven)
    dpost/dt  =   -post/(tc_post)       : 1 (event-driven)
'''
stdp_pre = '''
    w = clip(w + nu_pre*post, -wmax, wmax) * int(post>0.3*Apost) + clip(w - nu_pre*post, -wmax, wmax) * int(post<=0.3*Apost);
    pre += Apre;
'''
stdp_pre_e = stdp_pre + 'g_e_post += w;'
stdp_pre_i = stdp_pre + 'g_i_post += w;'
stdp_post  = 'w = clip(w + nu_post * pre, 0, wmax); post += Apost;'

In [ ]:
# stimuli_on, stimuli_off = load_stimuli('./pic_60_1.tif')

In [ ]:
neuron_groups = {}
neuron_groups_list = [
    ('L4_i_NF', 'i'), ('L4_NF', 'e'), ('L4_i_FN', 'i'), ('L4_FN', 'e'),
    ('L4', 'e'), 
    ('L3', 'e'), 
    ('L2', 'i')
]

for name, e_i in neuron_groups_list:
    if (e_i == 'e'):
        neuron_groups[name] = NeuronGroup(n_L4, neuron_eqs_e, threshold=v_thresh_e_str, refractory=refrac_e, reset=scr_e, method='euler')
        neuron_groups[name].v    = v_rest_e - 40.*mV 
        neuron_groups[name].theta = np.ones((n_L4)) * 20.0*mV
    elif (e_i == 'i'):
        neuron_groups[name] = NeuronGroup(n_L4, neuron_eqs_i, threshold=v_thresh_i_str, refractory=refrac_i, reset=v_reset_i_str, method='euler')
        neuron_groups[name].v    = v_rest_i - 40.*mV 

input_groups = {}
input_groups['LGN_on']   = PoissonGroup(n_LGN, 0*Hz)
input_groups['LGN_off']  = PoissonGroup(n_LGN, 0*Hz)

gc.collect()

1498

In [ ]:
weight_path = initial_path
# weight_path = saved_path
connections = {}
#[name, shape, weight_file, source, target, equation_type]

input_neuron_conn = [
    ['LGN_on_L4_NF', (n_LGN, n_L4), 'LGN_L4_NF.npy', 'LGN_on', 'L4_NF', 'e'],
    ['LGN_off_L4_NF', (n_LGN, n_L4), 'LGN_L4_FN.npy', 'LGN_off', 'L4_NF', 'e'],
    ['LGN_off_L4_i_NF', (n_LGN, n_L4), 'LGN_L4_NF.npy', 'LGN_off', 'L4_i_NF', 'e'],
    ['LGN_on_L4_i_NF', (n_LGN, n_L4), 'LGN_L4_FN.npy', 'LGN_on', 'L4_i_NF', 'e'],
    
    ['LGN_off_L4_FN', (n_LGN, n_L4), 'LGN_L4_NF.npy', 'LGN_off', 'L4_FN', 'e'],
    ['LGN_on_L4_FN', (n_LGN, n_L4), 'LGN_L4_FN.npy', 'LGN_on', 'L4_FN', 'e'],
    ['LGN_on_L4_i_FN', (n_LGN, n_L4), 'LGN_L4_NF.npy', 'LGN_on', 'L4_i_FN', 'e'],
    ['LGN_off_L4_i_FN', (n_LGN, n_L4), 'LGN_L4_FN.npy', 'LGN_off', 'L4_i_FN', 'e'],
]

neuron_neuron_conn = [
    ['L4_i_L4_NF', (n_L4, n_L4), 'L4_L4.npy', 'L4_i_NF', 'L4_NF', 'i', False],
    ['L4_i_L4_FN', (n_L4, n_L4), 'L4_L4.npy', 'L4_i_FN', 'L4_FN', 'i', False],
    ['L4_NF_L4', (n_L4, n_L4), 'L4_L4.npy', 'L4_NF', 'L4', 'e', False],
    ['L4_FN_L4', (n_L4, n_L4), 'L4_L4.npy', 'L4_FN', 'L4', 'e', False],
    
    ['L4_L3', (n_L4, n_L3), 'L4_L3.npy', 'L4', 'L3', 'e', False],
    ['L3_L3', (n_L3, n_L3), 'L3_L3.npy', 'L3', 'L3', 'e', True],
    ['L3_L2', (n_L3, n_L2), 'L3_L2.npy', 'L3', 'L2', 'e', False],
    ['L2_L3', (n_L2, n_L3), 'L2_L3.npy', 'L2', 'L3', 'i', False],
    
    ['L3_L4', (n_L3, n_L4), 'L3_L4.npy', 'L3', 'L4', 'e', False]
]


for name, shape, weight_file, source, target, _ in input_neuron_conn:
    weightMatrix = get_matrix_from_file(weight_path + weight_file, shape)
    connections[name]= Synapses(input_groups[source], neuron_groups[target], model=model, on_pre=pre_e, on_post=post)
    connections[name].connect(True)
    connections[name].w = weightMatrix[connections[name].i, connections[name].j]
#     connections[name].delay = 'minDelay + rand() * deltaDelay'
    
    
for name, shape, weight_file, source, target, equation, learn in neuron_neuron_conn:
    if (learn):
        model_eq = stdp_model
        post_eq = stdp_post
        if (equation == 'e'):
            pre_eq = stdp_pre_e
        elif (equation == 'i'):
            pre_eq = stdp_pre_i
    else:
        model_eq = model
        post_eq = post
        if (equation == 'e'):
            pre_eq = pre_e
        elif (equation == 'i'):
            pre_eq = pre_i
    
    weightMatrix = get_matrix_from_file(weight_path + weight_file, shape)
    connections[name]= Synapses(neuron_groups[source], neuron_groups[target], model=model_eq, on_pre=pre_eq, on_post=post_eq)
    connections[name].connect(True)
    connections[name].w = weightMatrix[connections[name].i, connections[name].j]
    
gc.collect()

WARNING    Cannot use Cython, a test compilation failed: Microsoft Visual C++ 14.0 is required. Get it with "Microsoft Visual C++ Build Tools": https://visualstudio.microsoft.com/downloads/ (DistutilsPlatformError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]


1816

In [ ]:
spike_monitors = {}

spike_monitors['LGN_on']      = SpikeMonitor(input_groups['LGN_on'])
spike_monitors['LGN_off']     = SpikeMonitor(input_groups['LGN_off'])

for name,_ in neuron_groups_list:
    spike_monitors[name]   = SpikeMonitor(neuron_groups[name])

In [ ]:
net = Network()
for obj_list in [neuron_groups, input_groups, connections, spike_monitors]:
    for key in obj_list:
        net.add(obj_list[key])

In [ ]:
plot_lateral_distribution('L3_L3')

hh 17907.516484911746 of 100226.45430962848 : 17.86705576712337
hv 2573.3042967182314 of 100226.45430962848 : 2.567490104726793
hd1 2572.821246114551 of 100226.45430962848 : 2.5670081455404605
hd2 2573.2572132577184 of 100226.45430962848 : 2.5674431276478993
vh 2573.249996765746 of 100226.45430962848 : 2.5674359274610605
vv 17916.2149873062 of 100226.45430962848 : 17.875734615890767
vd1 2573.301424170143 of 100226.45430962848 : 2.567487238669016
vd2 2572.9276032635903 of 100226.45430962848 : 2.567114262383336
d1h 2572.7545500989036 of 100226.45430962848 : 2.56694160021956
d1v 2572.7912555331986 of 100226.45430962848 : 2.5669782227206235
d1d1 16760.36879939859 of 100226.45430962848 : 16.72249997752187
d1d2 2573.0884863048345 of 100226.45430962848 : 2.567274781921169
d2h 2572.671459533175 of 100226.45430962848 : 2.56685869739086
d2v 2573.4665332263776 of 100226.45430962848 : 2.5676519746734687
d2d1 2572.7798938248834 of 100226.45430962848 : 2.5669668866832533
d2d2 16765.940079199216 of 1

In [ ]:
plot_preferred_distribution()

In [ ]:
# load stimuli files
stimuli_on, stimuli_off = get_stimuli(stimuli_file_path, 5, 20, 10)

# TRAIN

In [ ]:
previous_spike_count = np.zeros(n_L4)
net.run(resting_time)

j = 0
while j < (num_epochs):
    spike_rates_on = stimuli_on.reshape((n_LGN)) / 8. * input_intensity
    spike_rates_off = stimuli_off.reshape((n_LGN)) / 8. * input_intensity

    input_groups['LGN_on'].rates = spike_rates_on * Hz
    input_groups['LGN_off'].rates = spike_rates_off * Hz

    print('run example number:', j+1, 'of', num_epochs)
    net.run(single_example_time, report='text')   # 0.35 s

    current_spike_count = np.asarray(spike_monitors['L4'].count[:]) - previous_spike_count
    previous_spike_count = np.copy(spike_monitors['L4'].count[:])

    if np.sum(current_spike_count) < 1:
        if (input_intensity == 5):
            break;
        print("F - spike count", np.sum(current_spike_count))
        input_intensity += 1 
    else:     
        print("S - spike count", np.sum(current_spike_count))
        input_intensity = start_input_intensity
        j += 1
        
    input_groups['LGN_on'].rates = 0 * Hz
    input_groups['LGN_off'].rates = 0 * Hz
    net.run(resting_time)

    gc.collect()

In [ ]:
c_shape = (c_length, c_length, orientations)
r_shape = (r_length, r_length)
fig, axarr = subplots(2,5, figsize = (15, 7))

data = np.copy(spike_monitors['L3'].count).reshape(c_shape).transpose()
# q80 = np.percentile(data, 50)
# data = np.where((data < q80), 0, data)

v_min = data.min(); v_max = data.max()
print('L3 v_min', v_min, 'v_max', v_max)

im3 = axarr[0,0].imshow(data[h_index].transpose(), vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[0,0].title.set_text('L3_h'); 
plt.colorbar(im3, ax=axarr[0,0])
im3 = axarr[1,0].imshow(data[v_index].transpose(), vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[1,0].title.set_text('L3_v'); 
plt.colorbar(im3, ax=axarr[1,0])
im3 = axarr[0,1].imshow(data[d1_index].transpose(), vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[0,1].title.set_text('L3_d1');
plt.colorbar(im3, ax=axarr[0,1])
im3 = axarr[1,1].imshow(data[d2_index].transpose(), vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[1,1].title.set_text('L3_d2');
plt.colorbar(im3, ax=axarr[1,1])


data = np.copy(spike_monitors['L4'].count).reshape(c_shape).transpose()
# q80 = np.percentile(data, 50)
# data = np.where((data < q80), 0, data)

v_min = data.min(); v_max = data.max()
print('L4 v_min', v_min, 'v_max', v_max)

im4 = axarr[0,2].imshow(data[h_index].transpose(), vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[0,2].title.set_text('L4_h')
plt.colorbar(im4, ax=axarr[0,2])
im4 = axarr[1,2].imshow(data[v_index].transpose(), vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[1,2].title.set_text('L4_v')
plt.colorbar(im4, ax=axarr[1,2])
im4 = axarr[0,3].imshow(data[d1_index].transpose(), vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[0,3].title.set_text('L4_d1')
plt.colorbar(im4, ax=axarr[0,3])
im4 = axarr[1,3].imshow(data[d2_index].transpose(), vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[1,3].title.set_text('L4_d2')
plt.colorbar(im4, ax=axarr[1,3])


data = np.copy(spike_monitors['LGN_on'].count).reshape(r_shape)
v_min = data.min(); v_max = data.max()
print('LGN on', 'max', v_max, 'min', v_min)
axarr[0,4].imshow(data, vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[0,4].title.set_text('LGN_on')

data = np.copy(spike_monitors['LGN_off'].count).reshape(r_shape)
v_min = data.min(); v_max = data.max()
print('LGN off', 'max', v_max, 'min', v_min)
axarr[1,4].imshow(data, vmax=v_max, vmin=v_min, cmap = cmap.get_cmap('binary'))
axarr[1,4].title.set_text('LGN_off')

In [ ]:
fig, axarr = subplots(1,2, figsize = (12,6))

d_L3 = np.copy(spike_monitors['L3'].count).reshape(c_shape).transpose()
# q80 = np.percentile(d_L3, 95)
# d_L3 = np.where((d_L3 < q80), 0, d_L3)

d_L4 = np.copy(spike_monitors['L4'].count).reshape(c_shape).transpose()
# q80 = np.percentile(d_L4, 95)
# d_L4 = np.where((d_L4 < q80), 0, d_L4)

f_L3 = np.zeros((c_length,c_length))
f_L4 = np.zeros((c_length,c_length))
for o_index in range(orientations):
    f_L3 += d_L3[o_index].transpose()
    f_L4 += d_L4[o_index].transpose()

im_L3 = axarr[0].imshow(f_L3, cmap = cmap.get_cmap('binary'))
axarr[0].title.set_text('L3');
im_L4 = axarr[1].imshow(f_L4, cmap = cmap.get_cmap('binary'))
axarr[1].title.set_text('L4');

print('L4', d_L4.max())
print('L3', d_L3.max())

In [ ]:
gc.collect()

In [ ]:
bend_hv = [(7,1), (6,1), (5,1), (4,1), (3,1), (2,1), (1,1), (1,2), (1,3), (1,4), (1,5), (1,6), (1,7)]
plot_integration(bend_hv)

bend_h = [(1,4), (1,5), (1,6), (1,7), (1,8), (1,9), (1,10), (1,11), (1,12), (1,13), (1,14), (1,15)]
plot_integration(bend_h)

bend_v = [(5,18), (6,18), (7,18), (8,18), (9,18), (10,18), (11,18), (12,18), (13,18), (14,18)]
plot_integration(bend_v)

bend_dh = [(14,6), (15,7), (16,8), (17,9), (18,10), (18,11), (18,12), (18,13), (18,14), (18,15)]
plot_integration(bend_dh)

In [ ]:
# plot_integration(18, v_index)
# plot_integration(1, v_index)
# plot_integration(18, h_index)
# plot_integration(1, h_index)

In [ ]:
# plot_lateral_distribution('L3_L3')

In [ ]:
# save connections
# save_connections()

In [ ]:
# plot(spike_monitors['L4'].t/ms, spike_monitors['L4'].i, '.k')
# xlabel('Time (ms)')
# ylabel('Neuron index');

In [ ]:
# plot(spike_monitors['L3'].t/ms, spike_monitors['L3'].i, '.k')
# xlabel('Time (ms)')
# ylabel('Neuron index');

In [ ]:
# plot(spike_monitors['L2'].t/ms, spike_monitors['L2'].i, '.k')
# xlabel('Time (ms)')
# ylabel('Neuron index');

In [ ]:
# plot_preferred_distribution()

In [ ]:
get_matrix_from_file('./initial_weights/LGN_L4_NF.npy', (n_LGN, n_L4))